In [1]:
import os
import psycopg2 as pg
import pandas as pd
from dotenv import find_dotenv, load_dotenv

In [2]:
#Getting credentials from the PostgreSQL

dotenv_path = find_dotenv()

load_dotenv(dotenv_path)

host = os.getenv("host")
port = os.getenv("port")
database = os.getenv("database")
username = os.getenv("username")
password = os.getenv("password")

In [3]:
conn = pg.connect(user=username, password=password, host=host, port=port, database=database) 

# 1.0 SQUEMA QUERY

In [4]:
cursor = conn.cursor()

query_squema = """
    SELECT nspname
    FROM pg_catalog.pg_namespace
"""

cursor.execute(query_squema)

record = cursor.fetchall()
cursor.close()
conn.close()

# 2.0 TABLE QUERY

In [8]:
cursor = conn.cursor()

query_tables = """
    SELECT tablename
    FROM pg_tables
    WHERE schemaname = 'pa004'
"""

cursor.execute(query_tables)

record = cursor.fetchall()
print(record)

cursor.close()
conn.close()

InterfaceError: connection already closed

# 3.0 COLLECTING DATA USING PSYCOPG2

In [10]:
cursor = conn.cursor()

query_tables_users = """
    SELECT *
    FROM pa004.users u
    WHERE u.age > 44
    LIMIT 10
"""

cursor.execute(query_tables_users)

record = cursor.fetchall()
print(record)

cursor.close()
conn.close()

[(2, 'Male', 76, 3.0, 26.0), (3, 'Male', 47, 28.0, 26.0), (8, 'Female', 56, 28.0, 26.0), (11, 'Female', 47, 35.0, 124.0), (14, 'Male', 76, 28.0, 13.0), (15, 'Male', 71, 28.0, 30.0), (20, 'Female', 60, 33.0, 124.0), (21, 'Male', 65, 28.0, 124.0), (22, 'Male', 49, 28.0, 124.0), (27, 'Female', 51, 28.0, 124.0)]


In [11]:
data = pd.DataFrame(record)
data.head()

,0,1,2,3,4
0,2,Male,76,3.0,26.0
1,3,Male,47,28.0,26.0
2,8,Female,56,28.0,26.0
3,11,Female,47,35.0,124.0
4,14,Male,76,28.0,13.0


# 4.0 COLLECTING DATA USING PANDAS

In [12]:
import pandas.io.sql as psql

In [13]:
query_tables_users = """
    SELECT *
    FROM pa004.users u
    WHERE u.age > 44
    LIMIT 10
"""

df = psql.read_sql(query_tables_users, conn)
df.head()

/tmp/ipykernel_10406/3444200819.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = psql.read_sql(query_tables_users, conn)


InterfaceError: connection already closed

In [47]:
query_tables_users = """
    select *
    from pa004.users u left join pa004.vehicle v on (u.id = v.id)
				   left join pa004.insurance i on (u.id = i.id)
    where u.age > 44
"""

df1 = pd.read_sql(query_tables_users,conn)
df1.head()

/tmp/ipykernel_17642/2626664541.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql(query_tables_users,conn)


(131948, 14)

In [49]:
df1.head()

,id,gender,age,region_code,policy_sales_channel,id,driving_license,vehicle_age,vehicle_damage,id,previously_insured,annual_premium,vintage,response
0,8,Female,56,28.0,26.0,8,1,1-2 Year,Yes,8,0,32031.0,72,1
1,11,Female,47,35.0,124.0,11,1,1-2 Year,Yes,11,0,47576.0,46,1
2,39,Male,45,8.0,124.0,39,1,1-2 Year,Yes,39,0,42297.0,264,0
3,42,Female,54,28.0,122.0,42,1,1-2 Year,No,42,0,38560.0,184,0
4,53,Female,49,3.0,152.0,53,1,1-2 Year,No,53,1,25553.0,130,0


In [50]:
df1 = df1.drop(df1.columns[[5,9]], axis=1)
df1.head()

,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,Female,56,28.0,26.0,1,1-2 Year,Yes,0,32031.0,72,1
1,Female,47,35.0,124.0,1,1-2 Year,Yes,0,47576.0,46,1
2,Male,45,8.0,124.0,1,1-2 Year,Yes,0,42297.0,264,0
3,Female,54,28.0,122.0,1,1-2 Year,No,0,38560.0,184,0
4,Female,49,3.0,152.0,1,1-2 Year,No,1,25553.0,130,0
